In [7]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm_helper import *
import geopandas as gp
from osmread import Node, Way, Relation
import datetime
import pandas as pd
OSM_DC_BBOX_PATH = '../data/osm/osm_dc_bbox.osm'
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()
DC_OSM_IDS_PATH ='../data/osm/osm_ids_within_dc_polygon.txt'
DC_OSM_IDS={}
with open(DC_OSM_IDS_PATH) as f:
    DC_OSM_IDS = eval(f.readlines()[0])
for key, value in DC_OSM_IDS.items():
    print key, len(value)
    
    
def dc_obj(osm_objs, osm_ids, osmtype, osmtype_str):
    return [obj for obj in osm_objs.osm_objs[osmtype] if obj.id in osm_ids[osmtype_str]]

DC_NODES = dc_obj(OSM_DC_BBOX, DC_OSM_IDS,Node, 'Node')
DC_WAYS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Way, 'Way')
DC_RLTNS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Relation, 'Relation')
len(DC_WAYS), len(DC_NODES), len(DC_RLTNS)

begin reading osm 2016-11-12 20:55:31.984000
finish reading osm 2016-11-12 20:56:45.616000
["len of <class 'osmread.elements.Node'> = 2546260", "len of <class 'osmread.elements.Way'> = 306332", "len of <class 'osmread.elements.Relation'> = 3427"]
Node 1652906
Relation 2556
Way 194767


(194767, 1652906, 2556)

In [8]:
DC_NODES_WITH_TAG = filter_osm_data(DC_NODES)
DC_WAYS_WITH_TAG = filter_osm_data(DC_WAYS)
DC_RLTNS_WITH_TAG = filter_osm_data(DC_RLTNS)
len(DC_NODES_WITH_TAG), len(DC_WAYS_WITH_TAG), len(DC_RLTNS_WITH_TAG)


(37912, 190931, 2552)

In [12]:
import shapely.geometry as shpgeo

def lon_lats_closed(lon_lats):
    return lon_lats[0]==lon_lats[-1]

def way2lineOrpoly(osm_data, way):
    nodes = [osm_data.get_osm_node_by_id(nid) for nid in way.nodes]
    lon_lats = [(node.lon, node.lat) for node in nodes]
    return shpgeo.Polygon(lon_lats).buffer(0) if lon_lats_closed(lon_lats) and len(lon_lats)>3 else shpgeo.LineString(lon_lats)

def flatten_rltn_shpcltn(shpcltn):
    flat_shpcltn = []
    for shpobjs in shpcltn.values():
        flat_shpcltn.extend(shpobjs)
    return flat_shpcltn

def build_gpdf_list_from_osm(osm_data, nodes, ways, rltns):
    columns = ['id','type','geometry']
    shpobjs_from_nodes = [(node.id,'Node', node2pt(node)) for node in nodes]
    shpobjs_from_ways =  [(way.id,'Way', way2lineOrpoly(osm_data, way)) for way in ways]
    shpobjs_from_rltns = [(rltn.id, 'Relation', shpobj) for rltn in rltns for shpobj in flatten_rltn_shpcltn(rltn2cltn(osm_data,rltn))]
    print len(shpobjs_from_nodes),len(shpobjs_from_ways),len(shpobjs_from_rltns)
    shpobjs = []
    shpobjs.extend(shpobjs_from_nodes)
    shpobjs.extend(shpobjs_from_ways)
    shpobjs.extend(shpobjs_from_rltns)
    gpdf = gp.GeoDataFrame(shpobjs, columns=columns)
    return gpdf

In [7]:
gpdf = build_gpdf_list_from_osm(OSM_DC_BBOX, DC_NODES_WITH_TAG,DC_WAYS_WITH_TAG, DC_RLTNS_WITH_TAG)

37912 190931 7356


In [13]:
gpdf = build_gpdf_list_from_osm(OSM_DC_BBOX, DC_NODES_WITH_TAG,DC_WAYS_WITH_TAG, DC_RLTNS_WITH_TAG)

37912 190931 7356


In [8]:

opendc_segments_path = '../data/opendc_segments.geojson'
SEGMENTS = gp.read_file(opendc_segments_path)

In [14]:
def crs_prepossess(gpdf, init_crs, bfr_crs):
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs==None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)

def bfr_20m(seg):
    return seg.geometry.buffer(20)

def get_objs_nearby(segments, objs, bfr_func, init_crs, bfr_crs):
    seg_crs = crs_prepossess(segments, init_crs, bfr_crs)
    obj_crs = crs_prepossess(objs, init_crs, bfr_crs)
    seg_crs.geometry = seg_crs.apply(bfr_20m,axis=1)

    sjoin = gp.tools.sjoin(seg_crs, obj_crs, how='left',op='intersects')
    obj_ids_nearby = set(sjoin[~sjoin.index_right.isnull()].index_right)
    objs_nearby = objs[objs.index.isin(obj_ids_nearby)]
    return objs_nearby


In [15]:

OSM_near_DCSEG = get_objs_nearby(SEGMENTS, gpdf, bfr_20m, 4326, 3559)

In [19]:
OSM_near_DCSEG['type'].value_counts()

Way         140629
Node         19386
Relation      3920
dtype: int64

In [20]:
osm_ids = {}
for osmtype, grp in OSM_near_DCSEG.groupby('type'):
    ids = grp['id']
    unique_ids = set(ids)
    print len(ids), len(unique_ids)
    osm_ids[osmtype]=unique_ids


19386 19386
3920 2203
140629 140629


In [21]:
OSM_IDS_NEAR_OPENDC_SEGS_PATH = '../data/osm_ids_have_tag_near_opendc_segs.txt'
with open(OSM_IDS_NEAR_OPENDC_SEGS_PATH, 'w') as f:
    f.write(str(osm_ids))